<img src="./img/logo.png" align="right" width="50%"></img>
# iAtlantic GeoNode 101
### Tier 2: How to use OGC webservices offered by the iAtlantic GeoNode in your data analysis

## 1. Search through metadata using the OGC Catalogue Service (CSW)

The OWSLib library is a python library that makes accessing data and metadata through OGC services possible. The library abstracts all of the traditional HTTP API calls into programmatic function calls. We will be using this library throughout the tutorial to interact with the iAtlantic GeoNode OGC services.

The library is available from GitHub @  https://github.com/geopython/OWSLib

In [1]:
from owslib.csw import CatalogueServiceWeb

#### Create a CatalogueServiceWeb object

In [2]:
csw = CatalogueServiceWeb('http://www.geonode.iatlantic.eu/catalogue/csw')

#### Inspect its properties

In [3]:
print(csw.identification.type)
print(csw.identification.title)
print(csw.identification.version)
print([op.name for op in csw.operations])

CSW
GeoNode Catalogue
2.0.2
['GetCapabilities', 'DescribeRecord', 'GetDomain', 'GetRecords', 'GetRecordById', 'GetRepositoryItem']


#### Inspect the supported GetDomain parameters

In [4]:
csw.get_operation_by_name("GetDomain").parameters

{'ParameterName': {'values': ['DescribeRecord.outputFormat',
   'DescribeRecord.schemaLanguage',
   'DescribeRecord.typeName',
   'GetCapabilities.sections',
   'GetRecordById.ElementSetName',
   'GetRecordById.outputFormat',
   'GetRecordById.outputSchema',
   'GetRecords.CONSTRAINTLANGUAGE',
   'GetRecords.ElementSetName',
   'GetRecords.outputFormat',
   'GetRecords.outputSchema',
   'GetRecords.resultType',
   'GetRecords.typeNames']}}

#### Get suported constraint languages

In [5]:
csw.getdomain('GetRecords.CONSTRAINTLANGUAGE')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.CONSTRAINTLANGUAGE',
 'values': ['CQL_TEXT', 'FILTER']}

#### Get supported elementsets

In [6]:
csw.getdomain('GetRecords.ElementSetName')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.ElementSetName',
 'values': ['brief', 'full', 'summary']}

#### Get supported output formats

In [7]:
csw.getdomain('GetRecords.outputFormat')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.outputFormat',
 'values': ['application/json', 'application/xml']}

#### Get supported output schemas

In [8]:
csw.getdomain('GetRecords.outputSchema')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.outputSchema',
 'values': ['http://gcmd.gsfc.nasa.gov/Aboutus/xml/dif/',
  'http://www.interlis.ch/INTERLIS2.3',
  'http://www.isotc211.org/2005/gmd',
  'http://www.opengis.net/cat/csw/2.0.2',
  'http://www.opengis.net/cat/csw/csdgm',
  'http://www.w3.org/2005/Atom',
  'urn:oasis:names:tc:ebxml-regrep:xsd:rim:3.0']}

#### Search data by using OGC Filter Encoding

In [9]:
from owslib.fes import PropertyIsEqualTo, PropertyIsLike

>##### Example: search for AnyText fields that equal "bathymetry"

In [10]:
anytext_query = PropertyIsEqualTo('apiso:AnyText','bathymetry')
csw.getrecords2(constraints=[anytext_query], maxrecords=20, esn='full',outputschema='http://www.isotc211.org/2005/gmd')
print(csw.results)
for rec in csw.records:
    print(csw.records[rec].identification.title)
    print(csw.records[rec].identification.abstract)
    print("----")

{'matches': 4, 'returned': 4, 'nextrecord': 0}
Bathymetric Coverage iAtlantic
A dataset containing the outlines of Bathymetric surveys, relevant for the iAtlantic Consortium.
----
Bathymetric Surveys - AWI
A dataset containing the outlines of Bathymetric surveys, relevant for the iAtlantic Consortium.
----
Source references
Coverage of individual bathymetric surveys and Composite DTMs that contribute to the  EMODnet Bathymetry DTM.
----
Mean depth full coverage with land coverage
Mean depth based on source resolution of 1/16 arc minute (~125 meter). 
Full coverage ocean and land DTM in traditional atlas style colours.
----


C:\Users\timc\.conda\envs\geospatial\lib\site-packages\owslib\iso.py:116: FutureWarning: the .identification and .serviceidentification properties will merge into .identification being a list of properties.  This is currently implemented in .identificationinfo.  Please see https://github.com/geopython/OWSLib/issues/38 for more information
  warnings.warn(
C:\Users\timc\.conda\envs\geospatial\lib\site-packages\owslib\iso.py:543: FutureWarning: The .keywords and .keywords2 properties will merge into the .keywords property in the future, with .keywords becoming a list of MD_Keywords instances. This is currently implemented in .keywords2. Please see https://github.com/geopython/OWSLib/issues/301 for more information
  warnings.warn(


> ##### Example: search for title fields that contain "AWI"

In [11]:
title_query = PropertyIsLike('apiso:Title','%AWI%', wildCard="%")
csw.getrecords2(constraints=[title_query], maxrecords=20, esn='full',outputschema='http://www.isotc211.org/2005/gmd')
print(csw.results)
for rec in csw.records:
    print(csw.records[rec].identification.title)
    print(csw.records[rec].identification.abstract)
    print("----")

{'matches': 1, 'returned': 1, 'nextrecord': 0}
Bathymetric Surveys - AWI
A dataset containing the outlines of Bathymetric surveys, relevant for the iAtlantic Consortium.
----


#### Search with a CQL query

> Example: search for multibeam and iAtlantic in AnyText fields

In [12]:
csw.getrecords2(cql='csw:AnyText like "%multibeam%" AND csw:AnyText like "%iAtlantic%"',maxrecords=20,esn='full',outputschema='http://www.isotc211.org/2005/gmd')
print(csw.results)
for rec in csw.records:
    print(csw.records[rec].identification.title)
    print(csw.records[rec].identification.abstract)
    print("----")

{'matches': 2, 'returned': 2, 'nextrecord': 0}
Bathymetric Coverage iAtlantic
A dataset containing the outlines of Bathymetric surveys, relevant for the iAtlantic Consortium.
----
Bathymetric Surveys - AWI
A dataset containing the outlines of Bathymetric surveys, relevant for the iAtlantic Consortium.
----


#### Get record metadata

In [13]:
from pprint import pprint
records = [csw.records[rec] for rec in csw.records]
record = records[0]
pprint(vars(record.identification))

{'abstract': 'A dataset containing the outlines of Bathymetric surveys, '
             'relevant for the iAtlantic Consortium.',
 'abstract_url': None,
 'accessconstraints': [],
 'aggregationinfo': None,
 'alternatetitle': None,
 'bbox': <owslib.iso.EX_GeographicBoundingBox object at 0x0000024E49E83520>,
 'classification': [],
 'contact': [<owslib.iso.CI_ResponsibleParty object at 0x0000024E49E8BBB0>],
 'contributor': [],
 'creator': [<owslib.iso.CI_ResponsibleParty object at 0x0000024E49E8BB50>],
 'date': [<owslib.iso.CI_Date object at 0x0000024E49E53C70>],
 'datetype': [],
 'denominators': [],
 'distance': [],
 'edition': None,
 'extent': <owslib.iso.EX_Extent object at 0x0000024E49E83640>,
 'identtype': 'dataset',
 'keywords': [{'keywords': ['Bathymetry', 'features', 'Multibeam', 'Survey'],
               'thesaurus': {'date': None, 'datetype': None, 'title': None},
               'type': 'theme'},
              {'keywords': ['Global'],
               'thesaurus': {'date': None, 'da

#### Get record data

In [17]:
for resource in record.distribution.online:
    print('Description: '+resource.description)
    print('Protocol: '+ resource.protocol)
    print('URL: '+ resource.url)
    print("---")

Description: Online link to the 'Bathymetric Coverage iAtlantic' description on GeoNode
Protocol: WWW:LINK-1.0-http--link
URL: http://www.geonode.iatlantic.eu/layers/ialtantic_geonode_data:geonode:DCDB_200320_BathyCoverage_iAtlantic
---
Description: Bathymetric Coverage iAtlantic (Original Dataset Format)
Protocol: WWW:DOWNLOAD-1.0-http--download
URL: http://www.geonode.iatlantic.eu/download/34
---
Description: Bathymetric Coverage iAtlantic (JPEG Format)
Protocol: WWW:DOWNLOAD-1.0-http--download
URL: http://www.geonode.iatlantic.eu/geoserver/ows?layers=geonode%3ADCDB_200320_BathyCoverage_iAtlantic&service=WMS&format=image%2Fjpeg&request=GetMap&height=550&width=550&bbox=-88.310417175293000%2C-63.689586639404300%2C27.772916793823200%2C76.860420227050800&srs=EPSG%3A4326
---
Description: Bathymetric Coverage iAtlantic (PDF Format)
Protocol: WWW:DOWNLOAD-1.0-http--download
URL: http://www.geonode.iatlantic.eu/geoserver/ows?layers=geonode%3ADCDB_200320_BathyCoverage_iAtlantic&service=WMS&fo

### [>> Next: Visualize data using OGC Web Mapping Service (WMS)](./2_visualize_data_with_WMS.ipynb) 

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>